<a href="https://colab.research.google.com/github/shohdy-source/Machine-Learing-Deep-Learning-/blob/main/%20Financial%20Distress%20Prediction%20(Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import xgboost as xgb

from sklearn.metrics import r2_score
import io


In [1]:
from google.colab import files


uploaded = files.upload()

Saving Financial Distress.csv to Financial Distress.csv


In [4]:
data = pd.read_csv(io.BytesIO(uploaded['Financial Distress.csv']))
data

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,...,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
0,1,1,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,0.018854,0.182790,0.006449,0.85822,2.005800e+00,0.125460,6.97060,4.65120,0.050100,2.1984,0.018265,0.024978,0.027264,1.41730,9.5554,0.148720,0.66995,214.760,12.641,6.46070,0.043835,0.204590,0.35179,8.316100,0.28922,0.76606,2.5825,77.400,0.026722,1.630700,...,0.180160,1.50060,0.026224,7.05130,1174.90,5.33990,0.851280,12.837,0.061737,0.180900,209.87,-0.582550,0.471010,0.109900,0.000000,0.000000,0.22009,7.12410,15.38100,3.27020,17.8720,34.6920,30.087,12.8,7991.4,364.9500,15.8,61.476,4.0,36.0,85.437,27.07,26.102,16.000,16.0,0.2,22,0.060390,30,49
1,1,2,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,0.006423,0.035991,0.001795,0.85152,-4.864400e-01,0.179330,4.57640,3.75210,-0.014011,2.4575,0.027558,0.028804,0.041102,1.18010,7.2952,0.056026,0.67048,38.242,12.877,5.55060,0.265480,0.150190,0.41763,9.527600,0.41561,0.81699,2.6033,95.947,0.007580,0.837540,...,0.046857,1.00950,0.007864,4.60220,1062.50,3.73890,0.943970,12.881,-0.000565,0.056298,250.14,-0.474770,0.385990,0.369330,0.000000,0.000000,0.00000,7.41660,7.10500,14.32100,18.7700,124.7600,26.124,11.8,8322.8,0.1896,15.6,24.579,0.0,36.0,107.090,31.31,30.194,17.000,16.0,0.4,22,0.010636,31,50
2,1,3,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,-0.081423,-0.765400,-0.054324,0.89314,4.122000e-01,0.077578,11.89000,2.48840,0.028077,1.3957,0.012595,0.068116,0.014847,0.81652,7.1204,0.065220,0.84827,-498.390,13.225,16.25400,0.416570,0.074149,0.36723,9.351300,0.50356,0.91962,1.4931,144.670,-0.066483,0.955790,...,-0.579760,0.57832,-0.064373,11.98800,651.15,10.93400,0.934780,12.909,0.041625,0.047562,280.55,-1.000000,0.488440,0.053299,0.003785,0.005191,0.00000,3.63730,7.02130,1.15380,9.8951,6.4467,30.245,10.3,8747.0,11.9460,15.2,20.700,0.0,35.0,120.870,36.07,35.273,17.000,15.0,-0.2,22,-0.455970,32,51
3,1,4,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,-0.018807,-0.107910,-0.065316,0.89581,9.949000e-01,0.141120,6.08620,1.63820,0.093904,2.0588,0.011601,0.094385,0.014415,0.90391,7.9828,0.125160,0.80478,-75.867,13.305,8.89500,0.083774,0.054098,0.54360,7.090900,0.67133,0.93701,2.3533,219.750,-0.017000,0.383350,...,-0.150130,0.64508,-0.017731,6.11140,703.04,5.70280,0.874840,13.094,0.108400,0.101350,413.74,0.565000,0.344080,0.073356,0.000037,0.000045,0.00000,5.14420,9.90990,2.04080,-1.4903,-21.9070,34.285,11.5,9042.5,-18.7480,10.4,47.429,4.0,33.0,54.806,39.80,38.377,17.167,16.0,5.6,22,-0.325390,33,52
4,2,1,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,0.128030,0.577250,0.094075,0.81549,3.014700e+00,0.185400,4.39380,1.61690,0.239210,3.0311,0.006814,0.079346,0.008876,1.02510,4.7463,0.266020,0.76770,1423.100,11.575,17.48800,0.620770,0.046907,0.56963,9.486100,0.68143,0.94242,4.1296,222.650,0.131230,0.253010,...,0.607660,0.25782,0.131380,4.41510,2465.40,4.14080,0.733980,11.396,0.250310,0.222370,315.34,-0.060101,0.202420,1.229100,-0.002491,-0.002980,0.22688,7.12410,15.38100,3.27020,17.8720,34.6920,30.087,12.8,7991.4,364.9500,15.8,61.476,4.0,36.0,85.437,27.07,26.102,16.000,16.0,0.2,29,1.251000,7,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,422,10,0.438020,2.2605,0.202890,0.16037,0.18588,0.175970,0.198400,2.22360,1.091500,0.241640,0.226860,0.35580,1.550000e+07,0.839630,0.19101,12.09200,0.946730,8.1062,0.014077,0.000000,0.089439,0.52242,2.51

**Preprocessing**

In [5]:
print("Total missing values:", data.isna().sum().sum())


Total missing values: 0


In [6]:
data = data.drop(['Company', 'Time'], axis=1)


In [7]:
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [8]:
data = onehot_encode(data, column='x80', prefix='x80')


In [9]:
data


,Financial Distress,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,...,x81,x82,x83,x80_1,x80_2,x80_3,x80_4,x80_5,x80_6,x80_7,x80_8,x80_9,x80_10,x80_11,x80_12,x80_13,x80_14,x80_15,x80_16,x80_17,x80_18,x80_19,x80_20,x80_21,x80_22,x80_23,x80_24,x80_25,x80_26,x80_27,x80_28,x80_29,x80_30,x80_31,x80_32,x80_33,x80_34,x80_35,x80_36,x80_37
0,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,0.018854,0.182790,0.006449,0.85822,2.005800e+00,0.125460,6.97060,4.65120,0.050100,2.1984,0.018265,0.024978,0.027264,1.41730,9.5554,0.148720,0.66995,214.760,12.641,6.46070,0.043835,0.204590,0.35179,8.316100,0.28922,0.76606,2.5825,77.400,0.026722,1.630700,0.015016,0.005478,...,0.060390,30,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,0.006423,0.035991,0.001795,0.85152,-4.864400e-01,0.179330,4.57640,3.75210,-0.014011,2.4575,0.027558,0.028804,0.041102,1.18010,7.2952,0.056026,0.67048,38.242,12.877,5.55060,0.265480,0.150190,0.41763,9.527600,0.41561,0.81699,2.6033,95.947,0.007580,0.837540,0.027425,0.045434,...,0.010636,31,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,-0.081423,-0.765400,-0.054324,0.89314,4.122000e-01,0.077578,11.89000,2.48840,0.028077,1.3957,0.012595,0.068116,0.014847,0.81652,7.1204,0.065220,0.84827,-498.390,13.225,16.25400,0.416570,0.074149,0.36723,9.351300,0.50356,0.91962,1.4931,144.670,-0.066483,0.955790,0.017270,0.028059,...,-0.455970,32,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,-0.018807,-0.107910,-0.065316,0.89581,9.949000e-01,0.141120,6.08620,1.63820,0.093904,2.0588,0.011601,0.094385,0.014415,0.90391,7.9828,0.125160,0.80478,-75.867,13.305,8.89500,0.083774,0.054098,0.54360,7.090900,0.67133,0.93701,2.3533,219.750,-0.017000,0.383350,0.014327,0.203370,...,-0.325390,33,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,0.128030,0.577250,0.094075,0.81549,3.014700e+00,0.185400,4.39380,1.61690,0.239210,3.0311,0.006814,0.079346,0.008876,1.02510,4.7463,0.266020,0.76770,1423.100,11.575,17.48800,0.620770,0.046907,0.56963,9.486100,0.68143,0.94242,4.1296,222.650,0.131230,0.253010,0.008152,0.353010,...,1.251000,7,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,0.438020,2.2605,0.202890,0.16037,0.18588,0.175970,0.198400,2.22360,1.091500,0.241640,0.226860,0.35580,1.550000e+07,0.839630,0.19101,12.09200,0.946730,8.1062,0.014077,0.000000,0.089439,0.52242,2.5184,0.994340,0.15740,390.260,18.296,0.93687,-0.007698,0.002976,0.34999,0.013104,1.88290,0.98144,1431.7000,29.772,0.570230,0.003544,0.075736,0.362010,...,0.436380,4,41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3668,0.482410,1.9615,0.216440,0.20095,0.21642,0.203590,0.189870,1.93820,1.000100,0.270870,0.213610,0.38734,1.920000e+07,0.799050,0.25149,2.63990,0.940730,14.4280,0.018249,0.000000,0.092416,0.55872,2.7304,0.992440,0.19747,443.840,18.360,1.13980,0.066060,0.003484,0.38274,0.010072,1.76850,0.98267,1907.8000,136.370,0.558780,0.004359,0.084325,0.241230,...,0.438020,5,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3669,0.500770,1.7099,0.207970,0.26136,0.21399,0.193670,0.183890,1.68980,0.971860,0.281560,0.210970,0.44290,2.030000e+07,0.738640,0.35384,1.49010,0.905010,39.0520,0

In [10]:
print("Non-numeric columns:", len(data.select_dtypes('object').columns))


Non-numeric columns: 0


**Splitting/Scaling**

In [11]:
y = data['Financial Distress'].copy()
X = data.drop('Financial Distress', axis=1).copy()

In [12]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=100)

In [14]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

**Linear Regression**

In [15]:

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

print("Linear Regression R^2 Score:", lin_model.score(X_test, y_test))

Linear Regression R^2 Score: -1620.6397890018616


**Gradient Boosting Model**

In [16]:
params = {'learning_rate': 0.001, 'max_depth': 6, 'lambda': 0.01}

boost_model = xgb.train(
    params,
    dtrain,
    num_boost_round=10000,
    evals=[(dval, 'eval')],
    early_stopping_rounds=10,
    verbose_eval=False
)

print("XGB Model R^2 Score:", r2_score(y_test, boost_model.predict(dtest)))

XGB Model R^2 Score: 0.11591510038174635
